In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
from sklearn.cluster import KMeans
from scipy.stats.stats import pearsonr

In [ ]:
df = pd.read_csv('allPatientsKnnImputedData.csv',low_memory=False)
X = df.drop('Classes',axis=1)
features = X.columns
#X.replace(to_replace='?', value = 'NaN', inplace=True)
#imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
#X=imp.fit_transform(X)
Y = df['Classes']
X = pd.DataFrame(X,columns=features)

In [ ]:
#get covariance matrix, eigenvalues and eigenvectors
covMatrix  = X.cov()
w,v = np.linalg.eig(covMatrix)
wOrdered = sorted(w,reverse=True)


In [ ]:
#plot eigenvalues
f, axarr = plt.subplots(2, sharex=False)
axarr[0].plot(np.linspace(1,20,20),wOrdered[:20],'o')
axarr[0].set_title('Top 20 Eigenvalues')
#axarr[1].set_title('All Eigenvalues')
axarr[1].plot(np.linspace(1,len(wOrdered),len(wOrdered)),wOrdered,'o')
plt.show()

In [ ]:
wOrderedPercent =wOrdered/sum(wOrdered)
f, axarr = plt.subplots(2, sharex=False)
axarr[0].plot(np.linspace(1,20,20),wOrderedPercent[:20],'o')
axarr[0].set_title('Top 20 Eigenvalues')
axarr[0].set_ylabel('Percent variation explained by eigenvectors')
#axarr[1].set_ylabel('Percent variation explained by eigenvectors')
#axarr[1].set_title('All Eigenvalues')
axarr[1].plot(np.linspace(1,len(wOrderedPercent),len(wOrderedPercent)),wOrderedPercent,'o')
plt.show()                        

In [ ]:
eig_pairs = [(np.abs(w[i]), v[:,i]) for i in range(len(w))]
eig_pairs.sort(key=lambda x: x[0], reverse=True)
eigenvectorsSorted = [x[1] for x in eig_pairs]
eigenvectorsSorted[:4]

In [ ]:
def new_euclidean_distances(X, Y=None, Y_norm_squared=None, squared=False): 
    return pearsonr(X,Y)
KMeans.euclidean_distances = new_euclidean_distances 
results = np.zeros(18)
for i in range(0,18):
    eig = eigenvectorsSorted[:(i+2)]
    estimators = KMeans(n_clusters=2).fit(X.dot(np.asarray(eig).T))
    results[i] = max(sum(abs((estimators.labels_+1)- Y.as_matrix()))/len(Y),1-sum(abs((estimators.labels_+1)- Y.as_matrix()))/len(Y))

In [ ]:
plt.plot(np.linspace(2,18,18),results,'o')
plt.ylabel('Percent Accuracy')
plt.xlabel('Number of PCA variables')
plt.title('Accuracy vs. Number of PCA variables')

In [ ]:
plt.show()

In [ ]:
from sklearn.decomposition import PCA as sklearnPCA
results = np.zeros(18)
for i in range(0,18):
    sklearn_pca = sklearnPCA(n_components=(i+2))
    sklearn_transf = sklearn_pca.fit_transform(X)
    estimators = KMeans(n_clusters=2).fit(sklearn_transf)
    results[i] = max(sum(abs((estimators.labels_+1)- Y.as_matrix()))/len(Y),1-sum(abs((estimators.labels_+1)- Y.as_matrix()))/len(Y))

In [ ]:
plt.plot(np.linspace(2,18,18),results,'o')
plt.ylabel('Percent Accuracy')
plt.xlabel('Number of PCA variables')
plt.title('Accuracy vs. Number of PCA variables')
plt.show()